#  BÖLÜM 1: Veri Ön İşleme (Data Preprocessing)
**Amaç:** Ham veri setindeki kirliliği temizlemek, eksik verileri doldurmak ve modellemeye hazır hale getirip `islenmis_veri.csv` olarak kaydetmek.

In [1]:
import pandas as pd
import numpy as np
import os

# Veri Yükleme (Hata önleyici mod ile)
file_name = "ekip_odevi_ham_veri_30k.csv"

if os.path.exists(file_name):
    df = pd.read_csv(file_name, on_bad_lines='skip', encoding='utf-8')
    print(f" Ham Veri Yüklendi. Boyut: {df.shape}")
else:
    print(" HATA: Dosya bulunamadı. Lütfen dosya yolunu kontrol edin.")

 Ham Veri Yüklendi. Boyut: (30000, 17)


In [2]:
# Gereksiz Sütunları Atma
cols_to_drop = ['tax(£)', 'fuel type', 'engine size', 'mileage2', 'fuel type2', 'engine size2', 'reference']
df = df.drop(columns=cols_to_drop, errors='ignore')
print(" Gereksiz sütunlar temizlendi.")

 Gereksiz sütunlar temizlendi.


In [3]:
# Metin Temizliği ve Dönüşüm Fonksiyonu
def clean_currency(x):
    if isinstance(x, str):
        return x.replace('£', '').replace(',', '').strip()
    return x

# Temizliği uygula ve sayıya çevir
df['price'] = pd.to_numeric(df['price'].apply(clean_currency), errors='coerce')
df['mileage'] = pd.to_numeric(df['mileage'].apply(clean_currency), errors='coerce')

# Kritik eksikleri sil (Fiyatı veya KM'si olmayan araba işimize yaramaz)
df.dropna(subset=['price', 'mileage'], inplace=True)
print(f" Fiyat ve KM temizlendi. Güncel Satır Sayısı: {df.shape[0]}")

 Fiyat ve KM temizlendi. Güncel Satır Sayısı: 29732


In [4]:
# Eksik Değer Doldurma (Imputation)

# Sayısal sütunlar -> Medyan (Ortanca)
num_cols = ['year', 'tax', 'mpg', 'engineSize']
for col in num_cols:
    df[col] = df[col].fillna(df[col].median())

# Kategorik sütunlar -> Mod (En sık tekrar eden)
cat_cols = ['model', 'transmission', 'fuelType', 'marka']
for col in cat_cols:
    if col in df.columns:
        df[col] = df[col].fillna(df[col].mode()[0])

df['year'] = df['year'].astype(int)
print(" Eksik veriler dolduruldu.")

 Eksik veriler dolduruldu.


In [5]:
# Temiz veriyi kaydet
output_file = "islenmis_veri.csv"
df.to_csv(output_file, index=False)
print(f" Veri başarıyla '{output_file}' olarak kaydedildi.")

 Veri başarıyla 'islenmis_veri.csv' olarak kaydedildi.
